# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

# 1. ***Объявим набор входных параметров***:

In [21]:
import numpy as np
import yaml

In [ ]:
X = np.array([
    [0, 1],
    [1, 1],
    [1, 0],
    [-0.5, 0.5],
    [0, -0.5]
])

y = np.array([1, 1, 1, -1, -1])
theta0 = np.array([1.0, 1.0, 0.0])
lr = 0.1

# 2. ***Зададим функции $f(x_i, \theta)$ и $\mathcal{L}$($\theta$)***:

In [22]:
def f(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

def loss(X, y, theta):
    theta = np.asarray(theta)
    norm = (theta ** 2).sum()
    deltas = y * f(X, theta)
    return 0.1 * norm + np.mean(np.maximum(0, 1 - deltas))

# 3. ***Выведем результат до оптимизации***:

In [24]:
print("Prediction:", f(X, theta0))
print("Loss:", loss(X, y, theta0))

Prediction: [ 1.   2.   1.   0.  -0.5]
Loss: 0.5


# 4. ***Посчитаем градиент функции ошибок***:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Будем считать градиент относительно $\theta$.

***

### 1) ***Найдём градиент для $f(x, \theta)$***

$f(\theta_1)' = x_1,$

$f(\theta_2)' = x_2,$

$f(\theta_3)' = x_3.$

Следовательно, $\nabla f(x, \theta) = \begin{pmatrix} x_1 \\ x_2 \\ 1 \end{pmatrix}$

***

### 2) ***Найдём градиент для $\mathcal{L}(\theta)$***  

Функция ошибок $\mathcal{L}(\theta)$ состоит из двух частей.

*Для нахождения градиента посчитаем производные обеих частей*:


1. **Вычислим производные нормы $0.1||\theta||^2$**:

$||\theta||^2 = \theta_1^2 + \theta_2^2 + \theta_3^2$

 $\nabla 0.1||\theta||^2 = \begin{pmatrix} 0.2θ_1 \\ 0.2θ_2 \\ 0.2θ_3 \end{pmatrix}$


2. **Производная второй части**:

 $\xi_i = \begin{cases}
  0, & y_j \cdot φ_j(θ) > 1 \\
  -y_j x^i_j, & y_j \cdot φ_j(θ) < 1 \\
\end{cases}$
        
***

**Получаем:**

 $\mathfrak{L}(\theta)' = 0.2 \theta_i + \frac{1}{N} \sum_{j=1}^N \xi^i_j$

![image](https://user-images.githubusercontent.com/113284506/227794793-b1a403ec-f1a8-4484-81fb-42e0985fb6db.png)

# 5. ***Напишем код оптимизации***:

In [32]:
theta = theta0
def order(X, y, theta):
    der_theta = np.asarray(theta)
    deltas = y * f(X, der_theta)
    ext_der = (deltas < 1).astype(int) * (-y)
    der_mean = np.zeros_like(der_theta)
    der_mean[:2] = np.mean(ext_der[:, np.newaxis] * X, axis=0)
    der_mean[2] = np.mean(ext_der)
    return 0.1 * 2 * der_theta + der_mean

def gradient_descent(X, Y, theta, iterations=1000, lr=0.1):  
    for iter in range(iterations):
        theta -= order(X, Y, theta, lr)

# 6. ***Выведем результат после оптимизации***:

In [33]:
print("Prediction:", f(X, theta))
print("Loss:", loss(X, y, theta))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)

Prediction: [ 0.75218494  2.00436988  1.00436988 -0.37390753 -0.74781506]
Loss: 0.48815643632236133
